<a href="https://colab.research.google.com/github/leiyi-ihb/ColabPractics/blob/main/MachineLearningProject_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! if [ ! -f 'dataset_Li.pkl' ]; then wget http://bis.zju.edu.cn/nvwa/code_html/Dataset/Labels/Labels_Human.zip; else echo 'The data has been dowload'; fi

--2022-10-15 19:33:53--  http://bis.zju.edu.cn/nvwa/code_html/Dataset/Labels/Labels_Human.zip
Resolving bis.zju.edu.cn (bis.zju.edu.cn)... 210.32.159.96
Connecting to bis.zju.edu.cn (bis.zju.edu.cn)|210.32.159.96|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166656672 (159M) [application/zip]
Saving to: ‘Labels_Human.zip’

Labels_Human.zip    100%[===================>] 158.94M  8.32MB/s    in 29s     

2022-10-15 19:34:23 (5.53 MB/s) - ‘Labels_Human.zip’ saved [166656672/166656672]



# Machine Learning Project of http://bis.zju.edu.cn/nvwa/  
Cell Landscapes for Zebrafish, Drosophila, and Earthworm  
Cell landscapes constructed in Inferring predictive genetic models by deep learning of cross-species single-cell landscapes.

## 数据读取

In [25]:
import zipfile
import gzip
import pandas as pd

f=zipfile.ZipFile('Labels_Human.zip',"r")
f.extractall()
data = pd.read_csv('Labels_Human/Dataset.Human_Chrom8_train_test.cells.csv.gz', compression='gzip')
# print(len(data[:]))
data.iloc[0:1,0:1]



,Unnamed: 0
0,0


In [16]:
data.iloc[0:2, :]

,Unnamed: 0,Cell,Species,Celltype,Cluster
0,0,AdultAdrenalGland_2.CGAGTAGTCCCGTGCGGA-0,Human,Endothelial_Human11,Endothelial
1,1,AdultAdrenalGland_2.CGTATTTCAAAGTGATCA-0,Human,Endothelial_Human11,Endothelial


In [11]:
import pandas as pd
data = pd.read_csv(file)
# data.describe()

EmptyDataError: ignored

In [ ]:
import pickle
with open("dataset_Li.pkl", "rb") as f:
    data = pickle.load(f)
# confirmed the type of data
print(type(data))
# data is a Python dictionary with keys 'train', 'val', 'test'
print(type(data.keys()))
print(type(data['test']))
# # train has N=16000 datapoints for training the model
# print('train', data['train'][0].shape, data['train'][1].shape)
# # val has N=2000 datapoints for training the model
# print('val', data['val'][0].shape, data['val'][1].shape)
# # test has N=2000 datapoints for training the model
# print('test', data['test'][0].shape, data['test'][1].shape)

<class 'dict'>
<class 'dict_keys'>
<class 'tuple'>


## train_data and test_data

In [ ]:
x_train = data['test'][0]
# print(x_train[0])
x_label = data['test'][1]
y_test = data['train'][0]
y_label = data['train'][1]
x_train.shape # We have to check the data shap because following analysis need the 2d data
print(x_label.shape)

(2000, 1)


## change the 3d data into 2d

In [ ]:
nsample, nx, ny = x_train.shape
d2_x_train = x_train.reshape((nsample, nx*ny))
d2_x_train.shape
d2_y_test = y_test.reshape(y_test.shape[0], y_test.shape[1]*y_test.shape[2])

## data normalization

In [ ]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
d2_x_train = sc.fit_transform(d2_x_train)
d2_y_test = sc.fit_transform(d2_y_test)

## 适配SVM到训练集

In [ ]:
from sklearn.svm import SVC
classifer = SVC(kernel='linear', random_state=0)
classifer.fit(d2_x_train, x_label)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear', random_state=0)

## 预测测试集结果

In [ ]:
x_pred = classifer.predict(d2_x_train)
print(x_pred)

[1 1 1 ... 0 1 1]


## 创建混淆矩阵

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(x_label, x_pred)
cm

array([[ 988,    0],
       [   0, 1012]])

In [ ]:
x_test_set, x_label_set = d2_x_train, x_label
x_test_set

array([[-0.64764683, -0.49843701, -0.51091491, ..., -0.49217479,
         2.12359174, -0.66714819],
       [-0.64764683,  2.00627157, -0.51091491, ..., -0.49217479,
         2.12359174, -0.66714819],
       [ 1.54405141, -0.49843701, -0.51091491, ..., -0.49217479,
        -0.47090031, -0.66714819],
       ...,
       [-0.64764683, -0.49843701, -0.51091491, ..., -0.49217479,
        -0.47090031,  1.49891736],
       [-0.64764683, -0.49843701, -0.51091491, ...,  2.0317985 ,
        -0.47090031, -0.66714819],
       [-0.64764683, -0.49843701, -0.51091491, ..., -0.49217479,
        -0.47090031,  1.49891736]])

In [ ]:
x_test_tmp = x_test_set[:, 0:2]
# print(x_test_tmp)
x_label_tmp = x_label_set[:, 0]
x_label_tmp
print(len(x_test_tmp[:,0]))
# print(x_test_tmp[:,1][0:2])

2000


## 训练集结果可视化

In [ ]:
from pandas.io.formats.style import plt
import numpy as np
from matplotlib.colors import ListedColormap
# x_tmp_re1, x_label_set1 = x_tmp, x_label_tmp
X1, X2 = np.meshgrid(np.arange(start=x_test_tmp[:,0].min()-1, stop=x_test_tmp[:,0].max()+1),
                     np.arange(start=x_test_tmp[:,1].min()-1, stop=x_test_tmp[:,1].max()+1))
# print(x_test_tmp[:, 0])
# print(x_test_tmp[:, 1])
print(len(X1))
print(X2)
# y=classifer.predict(np.array([X1.ravel(), X2.ravel()]))
# y
plt.contourf(X1, X2, classifer.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1),
             alpha=0.75, cmap=ListedColormap(('red', 'green')))
# plt.xlim(x1.min(), x1.max())
# plt.ylim(x2.min(), x2.max())
# for i, j in enumerate(np.unique(x_label_set)):
#     plt.scatter(x_tmp_re[x_label_set == j, 0], x_tmp_re[x_label_set == j, 1],
#                 c=ListedColormap(('red', 'green'))(i), label=j)
# plt.title('SVM (Training set)')
# plt.xlabel('Age')
# plt.ylabel('Estimated Salary')
# plt.legend()
# plt.show()

5
[[-1.49843701 -1.49843701 -1.49843701 -1.49843701 -1.49843701]
 [-0.49843701 -0.49843701 -0.49843701 -0.49843701 -0.49843701]
 [ 0.50156299  0.50156299  0.50156299  0.50156299  0.50156299]
 [ 1.50156299  1.50156299  1.50156299  1.50156299  1.50156299]
 [ 2.50156299  2.50156299  2.50156299  2.50156299  2.50156299]]


ValueError: ignored